# A multi layer epithelium

Consisting of stacked 2D sheets

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import tyssue
from tyssue.core.sheet import Sheet
from tyssue.geometry.sheet_geometry import SheetGeometry
from tyssue.dynamics.sheet_vertex_model import SheetModel

In [2]:
base_specs = tyssue.config.geometry.sheet_spec()



In [21]:
specs

{'edge': {'dx': 0.0,
  'dy': 0.0,
  'dz': 0.0,
  'face': 0,
  'layer': 0,
  'length': 0.0,
  'nx': 0.0,
  'ny': 0.0,
  'nz': 1.0,
  'srce': 0,
  'trgt': 0},
 'face': {'area': 1.0,
  'is_alive': True,
  'layer': 0,
  'num_sides': 6,
  'perimeter': 1.0,
  'x': 0.0,
  'y': 0.0,
  'z': 0.0},
 'settings': {'geometry': 'cylindrical', 'height_axis': 'z'},
 'vert': {'basal_shift': 4.0,
  'height': 0.0,
  'is_active': True,
  'layer': 0,
  'rho': 0.0,
  'x': 0.0,
  'y': 0.0,
  'z': 0.0}}

In [76]:
specs = base_specs.copy()
specs['face']['layer'] = 0
specs['vert']['layer'] = 0
specs['vert']['depth'] = 0.0
specs['edge']['layer'] = 0
specs['settings']['geometry'] = 'flat'

In [23]:
from tyssue.core.generation import hexa_grid2d, from_2d_voronoi
from scipy.spatial import Voronoi

In [77]:
layer1_centers = hexa_grid2d(24, 24, 1, 1, 0.4)
layer2_centers = hexa_grid2d(16, 16, 2, 2, 1)
layer1_data = from_2d_voronoi(Voronoi(layer1_centers))
layer2_data = from_2d_voronoi(Voronoi(layer2_centers))


In [78]:
from tyssue.utils.utils import spec_updater

In [79]:
mod_specs = tyssue.config.dynamics.quasistatic_sheet_spec_anchors()
{'face': {'prefered_height': 1.,}}
layers_mod_specs = [
    {'face': {'prefered_height': 1.,
              'prefered_area': 1.},
    'vert': {'basal_shift':0.},
    },
    {'face': {'prefered_height': 1.,
              'prefered_area': 4.},
     'vert': {'basal_shift':0.},
    }]

for sheet, spec in zip(msheet, 
                       layers_mod_specs):
    
    spec_updater(mod_specs, spec)
    sheet.update_specs(SheetModel.dimentionalize(mod_specs),
                       reset=True)


In [81]:


class MultiSheet():
    
    
    def __init__(self, name, layer_datasets, specs):
        
        self.coords = ['x', 'y', 'z']
        self.layers = [Sheet('{}_{}'.format(name, i),
                             dsets, specs,
                             coords=self.coords)
                       for i, dsets in enumerate(layer_datasets)]
        for i, layer in enumerate(self):
            for dset in layer.datasets.values():
                dset['layer'] = i

    def __iter__(self):
        for layer in self.layers:
            yield layer
    
    def __getitem__(self, n):
        return self.layers[n]
    
    def __len__(self):
        return len(self.layers)
    
    @property
    def Nes(self):
        return [layer.Ne for i, layer in self]
    
    @property
    def Nvs(self):
        return [layer.Nv for i, layer in self]

    @property
    def Nfs(self):
        return [layer.Nf for i, layer in self]
    
    def concat_datasets(self):
        datasets = {}
        for key in ['edge', 'face', 'vert']:
            datasets[key] = pd.concat([layer.datasets[key]
                                       for layer in self])
        return datasets
    
    def update_interpolants(self):
        self.interpolant = [Rbf(sheet.vert_df['x'],
                                sheet.vert_df['y'],
                                sheet.vert_df['z'])
                            for sheet in self]


In [82]:
msheet = MultiSheet('two',
                    [layer1_data,
                     layer2_data],
                    specs)

In [83]:
from scipy.interpolate import Rbf


In [68]:
interp = Rbf(sheet.vert_df['x'],
             sheet.vert_df['y'],
             sheet.vert_df['z'],)

In [ ]:
interp()

In [70]:


class MultiSheetGeometry():
    
    @classmethod
    def update_all(cls, msheet):
        
        msheet.update_interpolants()
        for sheet in msheet:
            SheetGeometry.update_dcoords(sheet)
            SheetGeometry.update_length(sheet)
            SheetGeometry.update_centroid(sheet)
            SheetGeometry.update_normals(sheet)
            SheetGeometry.update_areas(sheet)
            SheetGeometry.update_perimeters(sheet)
            SheetGeometry.update_vol(sheet)
        cls.update_heights(msheet)

        
    @staticmethod
    def update_heights(msheet):
        
        for lower, upper in zip(msheet.interpolants[:-1], 
                                msheet[1:]):

            upper.vert_df['height'] = lower(upper.vert_df['x'],
                                            upper.vert_df['y'])
            
        for lower, upper in zip(msheet[:-1], 
                                msheet.interpolants[1:]):

            lower.vert_df['depth'] = upper(lower.vert_df['x'],
                                           lower.vert_df['y'])


In [72]:
MultiSheetGeometry.update_all(msheet)

In [ ]:
class MultiSheetModel():
    
    
    @staticmethod
    def compute_energy(msheet):
        
        E = 0
        for sheet in msheet:
            E += SheetModel.compute_energy(sheet)
            
    @staticmethod
    def interaction_energy(msheet):
        


2

In [16]:
%matplotlib qt
from tyssue.draw.plt_draw import sheet_view


fig, ax = plt.subplots()
draw_spec = {
    'vert':{
        'visible': False},
    'edge':{
        'head_width': 1e-3}
    }

for layer in msheet:
    
    fig, ax = sheet_view(layer, ax=ax, **draw_spec)

In [32]:
dz = 0.1
for i, layer in enumerate(msheet):
    layer.vert_df['z'] = dz * i
    

In [ ]:
msheet

In [ ]:
class MultiSheetGeom(SheetGeometry):
    
    
    